In [ ]:
! gdown --id 1hNb6f06NhjeSOtGkods0TMV_GD2Yl-8J

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1hNb6f06NhjeSOtGkods0TMV_GD2Yl-8J
To: /content/aptos_data_aug_upload.zip
100% 1.08G/1.08G [00:15<00:00, 69.3MB/s]


In [ ]:
from zipfile import ZipFile
file_name = "/content/aptos_data_aug_upload.zip"

with ZipFile(file_name,'r') as openZip:
  openZip.extractall()
  print('Done')

Done


In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

tf.random.set_seed(42)

datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255,
                                                          validation_split=0.2,
                                                          featurewise_center=True,
                                                          featurewise_std_normalization=True)

train_data = datagen.flow_from_directory(directory = '/content/aptos_data_aug_upload',
                                                  batch_size = 32,
                                                  subset = 'training',
                                                  target_size = (512, 512),
                                                  class_mode = 'categorical',
                                                  seed = 42)

valid_data = datagen.flow_from_directory(directory = '/content/aptos_data_aug_upload',
                                                 batch_size = 32,
                                                 subset = 'validation',
                                                 target_size = (512, 512),
                                                 class_mode = 'categorical',
                                                 seed = 42)

Found 4043 images belonging to 5 classes.
Found 1009 images belonging to 5 classes.


In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath='/content/EfficientNet/best_model1.h5',
                             monitor='val_accuracy',
                             mode='max',
                             save_best_only=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                              factor=0.1,
                              patience=5,
                              min_lr=0.00001)

In [ ]:
from tensorflow.keras.layers import Input, Dense, Flatten, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers.experimental import preprocessing

In [ ]:
data_aug_layer = Sequential([
    preprocessing.RandomFlip('horizontal'),
    preprocessing.RandomRotation(0.3),
    #preprocessing.RandomZoom(0.3),
    #preprocessing.RandomHeight(0.3),
    #preprocessing.RandomWidth(0.3)
    #preprocessing.Rescale(1./255)
], name = 'data_aug_layer')

In [ ]:
base_model = tf.keras.applications.EfficientNetV2B0(include_top = False)
base_model.trainable = True

for layer in base_model.layers[:-265]:
  layer.trainable = False

input = Input(shape = (512, 512, 3))
x = data_aug_layer(input, training = True)
x = base_model(x)
x = GlobalAveragePooling2D()(x)
x = Dense(10, activation = 'relu')(x)
x = Dense(10, activation = 'relu')(x)
x = Dense(5, activation = 'softmax')(x)

model1 = tf.keras.Model(inputs = input, outputs = x)

24274472/24274472 [==============================] - 2s 0us/step


In [ ]:
# base_model = tf.keras.applications.VGG19()
# base_model.trainable = True

In [ ]:
# for i,layer in enumerate(base_model.layers):
#   print(i, layer.name, layer.trainable)

In [ ]:
model1.compile(loss=tf.keras.losses.categorical_crossentropy,
               optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001), # when fine tuning you typically lower the learning rate by some amount
               metrics = ['accuracy'])

In [ ]:
# fine tune for another 15 epochs
fine_tune_epochs = 30
history1 = model1.fit(train_data,
                      epochs = fine_tune_epochs,
                      steps_per_epoch=len(train_data),
                      validation_data=valid_data,
                      validation_steps=len(valid_data),
                      callbacks = [checkpoint, reduce_lr])

/usr/local/lib/python3.10/dist-packages/keras/preprocessing/image.py:1861: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/preprocessing/image.py:1871: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(


Epoch 1/30
127/127 [==============================] - 149s 671ms/step - loss: 1.2380 - accuracy: 0.4954 - val_loss: 1.4628 - val_accuracy: 0.3786 - lr: 1.0000e-04
Epoch 2/30
127/127 [==============================] - 82s 649ms/step - loss: 1.0261 - accuracy: 0.5981 - val_loss: 1.3167 - val_accuracy: 0.5610 - lr: 1.0000e-04
Epoch 3/30
127/127 [==============================] - 81s 639ms/step - loss: 0.8695 - accuracy: 0.6315 - val_loss: 1.3932 - val_accuracy: 0.5411 - lr: 1.0000e-04
Epoch 4/30
127/127 [==============================] - 74s 584ms/step - loss: 0.7527 - accuracy: 0.6923 - val_loss: 1.5285 - val_accuracy: 0.5728 - lr: 1.0000e-04
Epoch 5/30
127/127 [==============================] - 73s 572ms/step - loss: 0.6265 - accuracy: 0.7984 - val_loss: 1.0466 - val_accuracy: 0.7037 - lr: 1.0000e-04
Epoch 6/30
127/127 [==============================] - 74s 577ms/step - loss: 0.5541 - accuracy: 0.8140 - val_loss: 0.5145 - val_accuracy: 0.8067 - lr: 1.0000e-04
Epoch 7/30
127/127 [=======

In [ ]:
import cv2 as cv2
import numpy as np

In [ ]:
def preprocessed_img(img_path):
  img = cv2.imread(img_path)
  img_t = cv2.addWeighted(img,4, cv2.GaussianBlur(img , (0,0) , 30) ,-4 ,128)
  img = img_t
  img = cv2.resize(img, (512, 512))
  h, w = img.shape[:2]
  cx, cy = w // 2, h // 2

  # Define the radius of the circle
  r = min(cx, cy)

  # Create a mask
  mask = np.zeros((h, w), np.uint8)
  cv2.circle(mask, (cx, cy), r, 1, -1)

  # Apply the mask to the image
  result = cv2.bitwise_and(img, img, mask=mask)

  # Save the cropped image or display it on screen
  return result

In [ ]:
test1 = preprocessed_img("/content/IDRiD_364.jpg")
test1 = test1/255.
output = model1.predict(tf.expand_dims(test1,axis = 0))

1/1 [==============================] - 0s 34ms/step


In [ ]:
output

array([[8.1281713e-04, 9.8165206e-04, 5.9955705e-02, 3.7471596e-03,
        9.3450266e-01]], dtype=float32)

In [ ]:
model1.save("Efficeinet.h5")

In [ ]:
tf.argmax(output[0])

<tf.Tensor: shape=(), dtype=int64, numpy=4>

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

tf.random.set_seed(4)

datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255,
                                                          validation_split=0.2,
                                                          featurewise_center=True,
                                                          featurewise_std_normalization=True)

train_data = datagen.flow_from_directory(directory = '/content/aptos_data_aug_upload',
                                                  batch_size = 32,
                                                  subset = 'training',
                                                  target_size = (512, 512),
                                                  class_mode = 'categorical',
                                                  seed = 42)

valid_data = datagen.flow_from_directory(directory = '/content/aptos_data_aug_upload',
                                                 batch_size = 32,
                                                 subset = 'validation',
                                                 target_size = (512, 512),
                                                 class_mode = 'categorical',
                                                 seed = 42)

Found 4043 images belonging to 5 classes.
Found 1009 images belonging to 5 classes.


In [ ]:
from tensorflow.keras.layers import Input, Dense, Flatten, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers.experimental import preprocessing

In [ ]:
data_aug_layer = Sequential([
    preprocessing.RandomFlip('horizontal'),
    preprocessing.RandomRotation(0.3),
    #preprocessing.RandomZoom(0.3),
    #preprocessing.RandomHeight(0.3),
    #preprocessing.RandomWidth(0.3)
    #preprocessing.Rescale(1./255)
], name = 'data_aug_layer')

In [ ]:
base_model = tf.keras.applications.EfficientNetV2B0(include_top = False)
base_model.trainable = True

for layer in base_model.layers[:-265]:
  layer.trainable = False

input = Input(shape = (512, 512, 3))
x = data_aug_layer(input, training = True)
x = base_model(x)
x = GlobalAveragePooling2D()(x)
x = Dense(5, activation = 'softmax')(x)

model1 = tf.keras.Model(inputs = input, outputs = x)

In [ ]:
model1.compile(loss=tf.keras.losses.categorical_crossentropy,
               optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001), # when fine tuning you typically lower the learning rate by some amount
               metrics = ['accuracy'])

In [ ]:
# fine tune for another 15 epochs
fine_tune_epochs = 30
history1 = model1.fit(train_data,
                      epochs = fine_tune_epochs,
                      steps_per_epoch=len(train_data),
                      validation_data=valid_data,
                      validation_steps=len(valid_data),
                      callbacks = [ reduce_lr])

/usr/local/lib/python3.10/dist-packages/keras/preprocessing/image.py:1861: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/preprocessing/image.py:1871: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(


Epoch 1/30
127/127 [==============================] - 117s 440ms/step - loss: 0.9478 - accuracy: 0.6342 - val_loss: 1.6236 - val_accuracy: 0.4202 - lr: 1.0000e-04
Epoch 2/30
127/127 [==============================] - 52s 411ms/step - loss: 0.5877 - accuracy: 0.7757 - val_loss: 2.0138 - val_accuracy: 0.4410 - lr: 1.0000e-04
Epoch 3/30
127/127 [==============================] - 52s 411ms/step - loss: 0.4930 - accuracy: 0.8066 - val_loss: 0.3604 - val_accuracy: 0.8801 - lr: 1.0000e-04
Epoch 4/30
127/127 [==============================] - 52s 412ms/step - loss: 0.4604 - accuracy: 0.8165 - val_loss: 0.5364 - val_accuracy: 0.8167 - lr: 1.0000e-04
Epoch 5/30
127/127 [==============================] - 52s 411ms/step - loss: 0.4013 - accuracy: 0.8429 - val_loss: 0.8217 - val_accuracy: 0.6898 - lr: 1.0000e-04
Epoch 6/30
127/127 [==============================] - 53s 413ms/step - loss: 0.3564 - accuracy: 0.8630 - val_loss: 0.4826 - val_accuracy: 0.8206 - lr: 1.0000e-04
Epoch 7/30
127/127 [=======

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

tf.random.set_seed(40)

datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255,
                                                          validation_split=0.2,
                                                          featurewise_center=True,
                                                          featurewise_std_normalization=True)

train_data = datagen.flow_from_directory(directory = '/content/aptos_data_aug_upload',
                                                  batch_size = 32,
                                                  subset = 'training',
                                                  target_size = (512, 512),
                                                  class_mode = 'categorical',
                                                  seed = 42)

valid_data = datagen.flow_from_directory(directory = '/content/aptos_data_aug_upload',
                                                 batch_size = 32,
                                                 subset = 'validation',
                                                 target_size = (512, 512),
                                                 class_mode = 'categorical',
                                                 seed = 42)

Found 4043 images belonging to 5 classes.
Found 1009 images belonging to 5 classes.


In [ ]:
base_model = tf.keras.applications.EfficientNetV2B0(include_top = False)
base_model.trainable = True

for layer in base_model.layers[:-265]:
  layer.trainable = False

input = Input(shape = (512, 512, 3))
x = data_aug_layer(input, training = True)
x = base_model(x)
x = GlobalAveragePooling2D()(x)
x = Dense(5, activation = 'softmax')(x)

model1 = tf.keras.Model(inputs = input, outputs = x)

In [ ]:
model1.compile(loss=tf.keras.losses.categorical_crossentropy,
               optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001), # when fine tuning you typically lower the learning rate by some amount
               metrics = ['accuracy'])

In [ ]:
# fine tune for another 15 epochs
fine_tune_epochs = 30
history1 = model1.fit(train_data,
                      epochs = fine_tune_epochs,
                      steps_per_epoch=len(train_data),
                      validation_data=valid_data,
                      validation_steps=len(valid_data),
                      callbacks = [ reduce_lr])

Epoch 1/30
127/127 [==============================] - 99s 423ms/step - loss: 0.9387 - accuracy: 0.6423 - val_loss: 1.2989 - val_accuracy: 0.4727 - lr: 1.0000e-04
Epoch 2/30
127/127 [==============================] - 52s 407ms/step - loss: 0.5768 - accuracy: 0.7791 - val_loss: 0.7432 - val_accuracy: 0.7364 - lr: 1.0000e-04
Epoch 3/30
127/127 [==============================] - 53s 413ms/step - loss: 0.4907 - accuracy: 0.8078 - val_loss: 0.8066 - val_accuracy: 0.6779 - lr: 1.0000e-04
Epoch 4/30
127/127 [==============================] - 52s 412ms/step - loss: 0.4367 - accuracy: 0.8316 - val_loss: 0.6485 - val_accuracy: 0.7582 - lr: 1.0000e-04
Epoch 5/30
127/127 [==============================] - 52s 411ms/step - loss: 0.3886 - accuracy: 0.8531 - val_loss: 0.4497 - val_accuracy: 0.8196 - lr: 1.0000e-04
Epoch 6/30
127/127 [==============================] - 53s 414ms/step - loss: 0.3545 - accuracy: 0.8667 - val_loss: 1.1193 - val_accuracy: 0.6848 - lr: 1.0000e-04
Epoch 7/30
127/127 [========

In [ ]:
import tensorflow as tf


In [ ]:
base_model = tf.keras.applications.VGG16(include_top = False)

58889256/58889256 [==============================] - 1s 0us/step


In [ ]:
for i in base_model.layers:
  print(i.trainable)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.applications import InceptionV3